# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Setup & Data Loading

In [ ]:
try:
  df = pd.read_csv('Amazon Sale Report.csv', encoding='latin1') # Try 'latin1'
except UnicodeDecodeError:
  try:
    df = pd.read_csv('Amazon Sale Report.csv', encoding='iso-8859-1') # Try 'iso-8859-1'
  except UnicodeDecodeError:
    df = pd.read_csv('Amazon Sale Report.csv', encoding='cp1252')
df.head()

# Data Overview & Cleaning


In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
del df['New']

In [ ]:
del df['PendingS']

In [ ]:
df.info()

In [ ]:
df['fulfilled-by']=df['fulfilled-by'].fillna("others")

In [ ]:
df['currency']=df['currency'].fillna("INR")

In [ ]:
df['Amount']=df['Amount'].fillna(np.random.choice(df['Amount']))

In [ ]:
df=df.dropna()

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df['Order ID'] = df['Order ID'].str.replace("S", "5", regex=False)

In [ ]:
df['Order ID'] = df['Order ID'].str.replace("-", "", regex=False).astype(int)

In [ ]:
df.info()

In [ ]:
df['Date'].value_counts()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], errors="coerce", infer_datetime_format=True)
df['Date']

In [ ]:
df.info()

In [ ]:
df["ship-postal-code"].value_counts()

In [ ]:
df["ship-postal-code"] = df["ship-postal-code"].fillna(0).astype(int)

In [ ]:
df.info()

In [ ]:
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Day"] = df["Date"].dt.day

In [ ]:
df.info()

# KPI

In [ ]:
import numpy as np
import pandas as pd

kpi = {}
kpi["date_range_start"] = df["Date"].min()
kpi["date_range_end"] = df["Date"].max()
kpi["total_orders"] = len(df)
kpi["total_units"] = df["Qty"].sum()
kpi["gross_revenue"] = df["Amount"].sum()
kpi["avg_order_value"] = (df["Amount"].sum() / df["Qty"].sum()) if df["Qty"].sum() else np.nan

# Cancellation/Return metrics if Status exists
if "Status" in df.columns:
    status_counts = df["Status"].value_counts(dropna=False)
    kpi["status_counts"] = status_counts.to_dict()
    # Consider these as "non-fulfilled" samples
    cancel_like = df["Status"].str.lower().isin(
        ["cancelled", "canceled", "return initiated", "returned"]
    )
    kpi["cancel_return_rate_%"] = round(cancel_like.mean() * 100, 2)
else:
    kpi["status_counts"] = {}
    kpi["cancel_return_rate_%"] = np.nan

# Convert KPI dict into a DataFrame
pd.DataFrame([kpi])

# Time-Series Trends

In [ ]:
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.to_period("M").astype(str)

ts_orders = df.groupby("Month").size().reset_index(name="Orders")
ts_units = df.groupby("Month")["Qty"].sum().reset_index(name="Units")
ts_revenue = df.groupby("Month")["Amount"].sum().reset_index(name="Revenue")

display(ts_orders.head())
display(ts_units.head())
display(ts_revenue.head())

# Plot Orders trend
plt.figure(figsize=(12,4))
plt.plot(ts_orders["Month"], ts_orders["Orders"], marker="o")
plt.title("Monthly Orders Trend")
plt.xlabel("Month")
plt.ylabel("Orders")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plot Revenue trend
plt.figure(figsize=(12,4))
plt.plot(ts_revenue["Month"], ts_revenue["Revenue"], marker="o")
plt.title("Monthly Revenue Trend")
plt.xlabel("Month")
plt.ylabel("Revenue")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
df.info()

# Category & Size Analysis

In [ ]:
import matplotlib.pyplot as plt

# Category summary
cat_summary = df.groupby("Category").agg(
    Orders=("Order ID", "count") if "Order ID" in df.columns else ("Qty", "size"),
    Units=("Qty", "sum"),
    Revenue=("Amount", "sum"),
    AvgSellingPrice=("Amount", "mean")
).sort_values("Revenue", ascending=False).head(15)

display(cat_summary)

plt.figure(figsize=(10,5))
plt.bar(cat_summary.index.astype(str), cat_summary["Revenue"])
plt.title("Top Categories by Revenue")
plt.xlabel("Category")
plt.ylabel("Revenue")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# Size analysis (if Size column exists)
if "Size" in df.columns:
    size_summary = df.groupby("Size").agg(
        Units=("Qty", "sum"),
        Revenue=("Amount", "sum")
    ).sort_values("Units", ascending=False).head(15)
    display(size_summary)

    plt.figure(figsize=(10,4))
    plt.bar(size_summary.index.astype(str), size_summary["Units"])
    plt.title("Top Sizes by Units Sold")
    plt.xlabel("Size")
    plt.ylabel("Units")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()


# Fulfillment Analysis

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

if "fulfilled-by" in df.columns:
    fulf_summary = df.groupby("Fulfilment").agg(
        Orders=("Order ID", "count") if "Order ID" in df.columns else ("Qty", "size"),
        Units=("Qty", "sum"),
        Revenue=("Amount", "sum"),
        AvgOrderValue=("Amount", lambda x: x.sum() / (len(x) if len(x) else np.nan))
    ).sort_values("Revenue", ascending=False)
    display(fulf_summary)

    plt.figure(figsize=(8,4))
    plt.bar(fulf_summary.index.astype(str), fulf_summary["Revenue"])
    plt.title("Revenue by Fulfillment Method")
    plt.xlabel("Fulfillment")
    plt.ylabel("Revenue")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

    # If courier status exists, explore delay signals
    if "Status" in df.columns:
        courier_summary = df.groupby("Status").agg(
            Orders=("Order ID", "count") if "Order ID" in df.columns else ("Qty", "size"),
            Units=("Qty", "sum"),
            Revenue=("Amount", "sum")
        ).sort_values("Orders", ascending=False)
        display(courier_summary)
else:
    print("Fulfilment column not present.")


# Geographic Analysis (State / City)

In [ ]:

# State-level performance
if "ship-state" in df.columns:
    state_summary = df.groupby("ship-state").agg(
        Orders=("Order ID", "count") if "Order ID" in df.columns else ("Qty", "size"),
        Units=("Qty", "sum"),
        Revenue=("Amount", "sum")
    ).sort_values("Revenue", ascending=False).head(20)
    display(state_summary)

    plt.figure(figsize=(10,5))
    plt.bar(state_summary.index.astype(str), state_summary["Revenue"])
    plt.title("Top States by Revenue")
    plt.xlabel("State")
    plt.ylabel("Revenue")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

# City-level performance
if "ship-city" in df.columns:
    city_summary = df.groupby("ship-city").agg(
        Orders=("Order ID", "count") if "Order ID" in df.columns else ("Qty", "size"),
        Units=("Qty", "sum"),
        Revenue=("Amount", "sum")
    ).sort_values("Revenue", ascending=False).head(20)
    display(city_summary)

    plt.figure(figsize=(10,5))
    plt.bar(city_summary.index.astype(str), city_summary["Revenue"])
    plt.title("Top Cities by Revenue")
    plt.xlabel("City")
    plt.ylabel("Revenue")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()


In [ ]:
# @title Revenue

from matplotlib import pyplot as plt
city_summary['Revenue'].plot(kind='line', figsize=(8, 4), title='Revenue')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# @title Units

from matplotlib import pyplot as plt
state_summary['Units'].plot(kind='line', figsize=(8, 4), title='Units')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# @title Orders

from matplotlib import pyplot as plt
state_summary['Orders'].plot(kind='line', figsize=(8, 4), title='Orders')
plt.gca().spines[['top', 'right']].set_visible(False)

# Actionable Insights

In [ ]:

insights = []

# Time windows with spikes (top 3 revenue months)
if "Month" in df.columns:
    top_months = df.groupby("Month")["Amount"].sum().sort_values(ascending=False).head(3)
    insights.append(f"Top revenue months: {', '.join([f'{m} (₹{v:,.0f})' for m, v in top_months.items()])}.")

# Category drivers
if "Category" in df.columns:
    top_cats = df.groupby("Category")["Amount"].sum().sort_values(ascending=False).head(3)
    insights.append("Top categories by revenue: " + ", ".join([f"{c} (₹{v:,.0f})" for c, v in top_cats.items()]) + ".")

# Size preferences
if "Size" in df.columns:
    top_sizes = df.groupby("Size")["Qty"].sum().sort_values(ascending=False).head(3)
    insights.append("Most sold sizes: " + ", ".join([f"{s} ({int(q)})" for s, q in top_sizes.items()]) + ".")

# Fulfillment
if "Fulfilment" in df.columns:
    fulf_rev = df.groupby("Fulfilment")["Amount"].sum().sort_values(ascending=False)
    best_fulfillment = fulf_rev.index[0] if len(fulf_rev) else None
    if best_fulfillment:
        insights.append(f"'{best_fulfillment}' drives the highest revenue (₹{fulf_rev.iloc[0]:,.0f}). Consider prioritizing this method.")

# Geographic opportunities
if "ship-state" in df.columns:
    top_states = df.groupby("ship-state")["Amount"].sum().sort_values(ascending=False).head(5)
    insights.append("Top states by revenue: " + ", ".join([f"{s} (₹{v:,.0f})" for s, v in top_states.items()]) + ".")

# Cancellation/returns
if "cancel_return_rate_%" in locals():
    pass  # handled in KPI, but echoed below
elif "Status" in df.columns:
    cancel_like = df["Status"].str.lower().isin(["cancelled", "canceled", "return initiated", "returned"])
    cr = (cancel_like.mean() * 100) if len(df) else 0
    insights.append(f"Cancellation/return rate: {cr:.2f}% — investigate SKUs and couriers linked to these outcomes.")

# Convert to numbered list
for i, line in enumerate(insights, 1):
    print(f"{i}. {line}")
